In [ ]:
import time
from google import genai

client = genai.Client(api_key="your_api_key")

In [ ]:
def generate_novel_segment(genre, theme, character_profile, full_novel, start_text=""):
    prompt = f"""
    Write a full-length {genre} novel that follows the traditional three-act structure. The story should have a clear beginning, middle,
    and end, with well-developed characters, conflicts, and resolutions. The characters must be properly introduced and developed throughout
    the story.

    1. **Beginning (Act 1)**: 
    - Introduce the protagonist, their environment, and their initial situation.
    - Establish the primary conflict or problem that the protagonist faces.
    - Present key supporting characters and their relationships with the protagonist.
    - Include a hook that makes the reader want to continue.
    
    2. **Middle (Act 2)**:
    - Develop the central conflict, showing the protagonist's struggles and growth.
    - Include rising action, obstacles, and challenges that complicate the situation.
    - Introduce subplots or side stories that complement the main narrative.
    - Show character development and decision-making, with moments of tension or reflection.
    
    3. **End (Act 3)**:
    - Resolve the central conflict in a satisfying way, whether through success or failure.
    - Provide a conclusion to the character arcs and subplots.
    - Include a denouement that wraps up any loose ends and leaves the reader with a sense of closure.
    - Ensure the ending feels natural and earned, not rushed or abrupt.

    The novel should include the theme of '{theme}', and the main character should be described as follows: {character_profile}.
    The novel should be at least 80,000 words, with consistent tone and style throughout.

    To counter the constraint of limited word count in response, multiple requests are being sent to the API to generate the novel segment.
    This request is one of the many requests that will be sent to the API to generate the novel segment. {len(full_novel.split())} have already
    been generated. {80000-len(full_novel.split())} words are left to be generated. Leave some short comments so the next iteration of
    of request can understand the context of the story so far. The comments should be enclosed within curly braces.
    The following text is some part of the novel generated so far:
    {start_text}
    Use this to understand how far the story has progressed and on what stage it is.
    Proper continuity and pace of the story must be ensured keeping in mind which part this is in the novel and how many words are left.
    """

    print("Sending request to Gemini API...")
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
        )
        
        if response.text:
            print("Novel segment generated successfully!")
            return response.text
        else:
            print("Error: No content generated.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
def print_novel_real_time(novel_text):
    print("Starting to write the novel in real-time...\n")
    for i in range(0, len(novel_text), 5):
        print(novel_text[i:i + 5], end="")
        time.sleep(0.0001)

def generate_full_novel(genre, theme, character_profile, total_words=80000):
    full_novel = ""
    current_text = ""

    while len(full_novel.split()) < total_words:
        print(f"Generating next section of the novel. Current length: {len(full_novel.split())} words")
        new_segment = generate_novel_segment(genre, theme, character_profile, full_novel, start_text=current_text)
        
        if new_segment:
            full_novel += new_segment
            current_text = new_segment[-500:]  
        else:
            print("Failed to generate new segment.")
            break

        time.sleep(1)

    return full_novel

In [ ]:
import os

if __name__ == "__main__":
    genre = "genre"
    theme = "theme"
    character_profile = "character profile"

    print(f"Generating a {genre} novel with the theme: '{theme}' and character profile: '{character_profile}'")

    total_words = 80000
    full_novel = ""
    current_text = ""

    while len(full_novel.split()) < total_words:
        print(f"Generating next section of the novel. Current length: {len(full_novel.split())} words")
        new_segment = generate_novel_segment(genre, theme, character_profile, full_novel, start_text=current_text)
        
        if new_segment:
            full_novel += new_segment
            current_text = new_segment[-500:]
        else:
            print("Failed to generate new segment.")
            break

        time.sleep(1)

    if full_novel:
        folder_path = "generated_novel"
        os.makedirs(folder_path, exist_ok=True)
        file_path = os.path.join(folder_path, "generated_novel.txt")
        
        with open(file_path, "w") as f:
            f.write(full_novel)
        print(f"\nNovel saved to '{file_path}'.")

In [ ]:
import pyttsx3

engine = pyttsx3.init()

with open('generated_novel/generated_novel.txt', 'r', encoding='utf-8') as file:
    novel_text = file.read()

def split_text(text, max_length=5000):
    chunks = []
    while len(text) > max_length:
        split_index = text.rfind('.', 0, max_length) + 1
        if split_index == 0:
            split_index = max_length
        chunks.append(text[:split_index])
        text = text[split_index:]
    chunks.append(text)
    return chunks

def text_to_audio(text_chunks, folder_path):
    for i, chunk in enumerate(text_chunks):
        file_path = os.path.join(folder_path, f'novel_part_{i+1}.mp3')
        engine.save_to_file(chunk, file_path)
        print(f'{file_path} saved successfully.')

text_chunks = split_text(novel_text)

text_to_audio(text_chunks, 'generated_novel')

engine.runAndWait()
